In [1]:
pip install git+https://github.com/hyperopt/hyperopt-sklearn.git

  Cloning https://github.com/hyperopt/hyperopt-sklearn.git to /tmp/pip-req-build-6iolv73c
  Running command git clone -q https://github.com/hyperopt/hyperopt-sklearn.git /tmp/pip-req-build-6iolv73c
  Created wheel for hpsklearn: filename=hpsklearn-0.0.3-py3-none-any.whl size=26922 sha256=160eb4a6db3e1ad826fe978195e55d396fcc03f541f26440cbaa5cb37e9fd4df
  Stored in directory: /tmp/pip-ephem-wheel-cache-f694l91b/wheels/47/a5/46/9ca750026db9dfa5de4bf4836194554cb0e2e01a245588ea59
Successfully built hpsklearn
Note: you may need to restart the kernel to use updated packages.


# Experiment 2.2 - Use hyperopt estimator on best training data from group

## Try hyperopt estimator on a transformed smote dataset

Charles has got the best model so far using standard scaler and smote to balance the classes. Try finding a new best base model with the same transformed data, and see if there is a better classifier out there thand logistic regression with liblinear.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import load_data function from helper file 
%load_ext autoreload
%autoreload 2

# fix system path
import sys
sys.path.append("/home/jovyan/work")

In [3]:
from src.features.helper_functions import load_sets

X_train, y_train, X_val, y_val, X_test = load_sets()

In [4]:
from hpsklearn import HyperoptEstimator
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing
from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [5]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [6]:
scaler = StandardScaler()

X_scale = scaler.fit_transform(X_train)

X_val_scale = scaler.transform(X_val)

print(X_scale)
print(X_val_scale)

[[ 0.07287296 -0.52299895 -0.6861803  ... -0.61178883 -0.33444528
  -0.76943971]
 [ 1.06917581  0.25148433  0.08196862 ... -0.12153046 -0.33444528
   0.05673496]
 [-0.74761174 -1.45462378 -1.36122026 ... -0.61178883 -0.48918032
  -1.32022282]
 ...
 [-0.45458149 -0.33218423 -0.9189527  ... -0.36665964 -0.0249752
  -0.63174393]
 [ 1.65523631  3.2932955   1.12944442 ...  0.61385711  0.43922991
   0.4698223 ]
 [ 0.24869111  0.88005048  1.22255338 ... -0.12153046  3.22446059
  -0.35635237]]
[[-1.21646014 -1.25258466 -1.08189338 ... -0.85691802 -0.48918032
  -1.32022282]
 [-0.22015729 -0.66891609 -1.0353389  ... -0.61178883 -0.17971024
  -0.35635237]
 [ 0.42450926 -0.27606225  1.36221682 ... -0.61178883  0.12975984
   0.19443074]
 ...
 [ 0.24869111  0.5882162   0.66389962 ...  1.10411549 -0.0249752
   0.88290964]
 [-1.09924804  0.18413796  0.0121369  ...  0.12359873  0.12975984
   0.05673496]
 [ 0.95196371  1.11576278  1.71137542 ...  1.34924467 -0.33444528
   2.3975632 ]]


In [7]:
# transform the dataset - will do automatic minority resampling
oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X_scale, y_train)

In [8]:
from collections import Counter
print('Resampled dataset shape %s' % Counter(y_smote))

Resampled dataset shape Counter({1: 5326, 0: 5326})


In [9]:
# define search
model = HyperoptEstimator(classifier=any_classifier('cla'), preprocessing = [], algo=tpe.suggest, max_evals=50, trial_timeout=30, seed = 42)
# pass an empty list to preprocessing to do nothing

# perform the search
model.fit(X_smote, y_smote)

# summarize performance
acc = model.score(X_val_scale, y_val) 
# hyperopt estimator only does accuracy or R2 scores https://github.com/hyperopt/hyperopt-sklearn/blob/fd718c44fc440bd6e2718ec1442b1af58cafcb18/hpsklearn/estimator.py#L844
print("Accuracy: %.3f" % acc)
# summarize the best model
print(model.best_model())

 94%|█████████▍| 16/17 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[06:26:36] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 95%|█████████▍| 18/19 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[06:27:36] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 97%|█████████▋| 35/36 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[06:28:58] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 98%|█████████▊| 47/48 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[06:29:36] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1607604574104/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
100%|██████████| 50/50 [00:00<00:00, 122.50trial/s, best loss: 0.002815579540122015]
Accuracy: 0.664
{'learner': KNeighborsClassifier(metric='euclidean', n_jobs=1, n_neighbors=1,
                     weights='distance'), 'preprocs': (), 'ex_preprocs': ()}


In [10]:
# Train this model again using the above model
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(metric='euclidean', n_jobs=1, n_neighbors=1,
                     weights='distance')

knc = model.fit(X_smote, y_smote)

In [11]:
from src.features.helper_functions import save_model
save_model(knc, 'knclf_scale_smote_2')

Model saved succesfully


In [10]:
# save new upsampled data to interim
np.save('../data/interim/X_train_scale_smote', X_smote)
np.save('../data/interim/y_train_scale_smote', y_smote)
np.save('../data/interim/X_val_scale', X_val_scale)

In [20]:

# model_smote = GradientBoostingClassifier(learning_rate=0.3200712536391533, max_depth=4,
#                            n_estimators=967, random_state=1,
#                            subsample=0.5066639200300195)

# gbc = model_smote.fit(X_smote_scale, y_smote)

In [ ]:
# predict class
y_train_preds = knc.predict(X_smote)
y_val_preds = knc.predict(X_val_scale)

In [ ]:
# predict proabilities
y_train_preds_prob = knc.predict_proba(X_smote)
y_val_preds_prob = knc.predict_proba(X_val_scale)

In [24]:
from src.features.helper_functions import result_metrics

In [26]:
result_metrics(y_smote, y_train_preds,y_train_preds_prob)

Accuracy: 100.00%
Precision: 100.00% 
Recall: 100.00% 
AUC using prediction probabilities: 100.000% 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5326
           1       1.00      1.00      1.00      5326

    accuracy                           1.00     10652
   macro avg       1.00      1.00      1.00     10652
weighted avg       1.00      1.00      1.00     10652

Confusion Matrix
[[5326    0]
 [   0 5326]]


In [27]:
result_metrics(y_val, y_val_preds,y_val_preds_prob)

Accuracy: 79.25%
Precision: 84.69% 
Recall: 91.88% 
AUC using prediction probabilities: 58.319% 
              precision    recall  f1-score   support

           0       0.24      0.13      0.17       257
           1       0.85      0.92      0.88      1343

    accuracy                           0.79      1600
   macro avg       0.54      0.53      0.53      1600
weighted avg       0.75      0.79      0.77      1600

Confusion Matrix
[[  34  223]
 [ 109 1234]]


In [25]:
# save model
from src.features.helper_functions import save_model

In [26]:
save_model(etclf, 'rez_etclf_1')

Model saved succesfully


In [ ]:
# predict on test set


In [27]:
# create an output for kaggle testing anyway.
y_test_preds = etclf.predict(X_test)
y_test_preds_prob = etclf.predict_proba(X_test)

In [28]:
print(y_test_preds)
print(y_test_preds_prob)

[0 0 1 ... 0 1 0]
[[0.55198684 0.44801316]
 [0.50364994 0.49635006]
 [0.3517363  0.6482637 ]
 ...
 [0.58697257 0.41302743]
 [0.32636914 0.67363086]
 [0.55008323 0.44991677]]


In [29]:
unique_elements, counts_elements = np.unique(y_test_preds, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[   0    1]
 [1619 2180]]


In [30]:
1619/2180

0.7426605504587156

In [31]:
from src.features.helper_functions import create_output

In [32]:
output = create_output(y_test_preds_prob)

In [33]:
output

,Id,TARGET_5Yrs
0,0,0.448013
1,1,0.496350
2,2,0.648264
3,3,0.689108
4,4,0.442623
...,...,...
3794,3794,0.623098
3795,3795,0.524807
3796,3796,0.413027
3797,3797,0.673631
